In [1]:
# Relevant imports
import numpy as np
import pandas as pd

# Extract MNIST csv data into train & test variables
train = np.array(pd.read_csv('train.csv', delimiter=','))
test = np.array(pd.read_csv('test.csv', delimiter=','))

# Check shape of train & test datasets
print(f'train shape: {train.shape}')
print(f'test shape: {test.shape}')

train shape: (42000, 785)
test shape: (28000, 784)


In [2]:
# Extract the first column of the training dataset into a label array
label = train[:,0]
# The train dataset now becomes all columns except the first
train = train[:,1:]

In [3]:
# Initialise vector of all zeroes with 10 columns and the same number of
# rows as the label array
Y = np.zeros((label.shape[0], 10))

# assign a value of 1 to each column index matching the label value
Y[np.arange(0,label.shape[0]), label] = 1.0

print(f'Label Value: {label[8]}')
print(f'Corresponding y vector: {Y[8]}')

Label Value: 5
Corresponding y vector: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [4]:
# Normalize test & training dataset
train = train / 255
test = test / 255

In [5]:
# Activation functions
def relu(X):
    return np.maximum(0,X)

# def softmax(X):
#     return np.exp(X)/sum(np.exp(X))

# stable softmax
def softmax(X):
    Z = X - max(X)
    numerator = np.exp(Z)
    denominator = np.sum(numerator)
    return numerator/denominator

In [13]:
# Calculates the output of a given layer
def calculate_layer_output(w, prev_layer_output, b, activation_type="relu"):
    # Steps 1 & 2
    g = w @ prev_layer_output + b

    # Step 3
    if activation_type == "relu":
        return relu(g)
    if activation_type == "softmax":
        return softmax(g)

In [14]:
# Initialize weights & biases
def init_layer_params(row, col):
    w = np.random.randn(row, col)
    b = np.random.randn(row, 1)
    return w, b

In [15]:
# Randomly initialize weights & baises for each layer
w1, b1 = init_layer_params(10, 784)
w2, b2 = init_layer_params(10, 10)
w3, b3 = init_layer_params(10, 10)

# Forward Pass through the Neural Network to make prediction
input = train[8:9].T # Make the input a single image vector
h1 = calculate_layer_output(w1, input, b1, activation_type="relu")
h2 = calculate_layer_output(w2, h1, b2, activation_type="relu")
output = calculate_layer_output(w3, h2, b3, activation_type="softmax")

print(f'Label Value: {label[8]}')
print(f'Prediction: {np.argmax(output)}')

Label Value: 5
Prediction: 4


In [16]:
# Calculate ReLU derivative
def relu_derivative(g):
    derivative = g.copy()
    derivative[derivative<=0] = 0
    derivative[derivative>0] = 1
    return np.diag(derivative.T[0])

# Calculate Softmax derivative
def softmax_derivative(o):
    derivative = np.diag(o.T[0])

    for i in range(len(derivative)):
        for j in range(len(derivative)):
            if i == j:
                derivative[i][j] = o[i] * (1 - o[i])
            else:
                derivative[i][j] = -o[i] * o[j]
    return derivative

In [17]:
def layer_backprop(previous_derivative, layer_output, previous_layer_output
                   , w, activation_type="relu"):
    # 1. Calculate the derivative of the activation func
    dh_dg = None
    if activation_type == "relu":
        dh_dg = relu_derivative(layer_output)
    elif activation_type == "softmax":
        dh_dg = softmax_derivative(layer_output)

    # 2. Apply chain rule to get derivative of Loss function with respect to:
    dL_dg = dh_dg @ previous_derivative # activation function

    # 3. Calculate the derivative of the linear function with respect to:
    dg_dw = previous_layer_output.T     # a) weight matrix
    dg_dh = w.T                         # b) previous layer output
    dg_db = 1.0                         # c) bias vector

    # 4. Apply chain rule to get derivative of Loss function with respect to:
    dL_dw = dL_dg @ dg_dw               # a) weight matrix
    dL_dh = dg_dh @ dL_dg               # b) previous layer output
    dL_db = dL_dg * dg_db               # c) bias vector

    return dL_dw, dL_dh, dL_db

In [18]:
def gradient_descent(w, b, dL_dw, dL_db, learning_rate):
    w -= learning_rate * dL_dw
    b -= learning_rate * dL_db
    return w, b

In [19]:
def get_prediction(o):
    return np.argmax(o)

In [20]:
# Compute Accuracy (%) across all training data
def compute_accuracy(train, label, w1, b1, w2, b2, w3, b3):
    # Set params
    correct = 0
    total = train.shape[0]

    # Iterate through training data
    for index in range(0, total):
        # Select a single data point (image)
        X = train[index: index+1,:].T

        # Forward pass: compute Output/Prediction (o)
        h1 = calculate_layer_output(w1, X, b1, activation_type="relu")
        h2 = calculate_layer_output(w2, h1, b2, activation_type="relu")
        o = calculate_layer_output(w3, h2, b3, activation_type="softmax")

        # If prediction matches label Increment correct count
        if label[index] == get_prediction(o):
            correct+=1

    # Return Accuracy (%)
    return (correct / total) * 100

In [21]:
# Set hyperparameter(s)
learning_rate = 0.01

# Set other params
epoch = 0
previous_accuracy = 100
accuracy = 0

# Randomly initialize weights & biases
w1, b1 = init_layer_params(10, 784)   # Hidden Layer 1
w2, b2 = init_layer_params(10, 10)    # Hidden Layer 2
w3, b3 = init_layer_params(10, 10)    # Output Layer

# While:
#  1. Accuracy is improving by 1% or more per epoch, and
#  2. There are 20 epochs or less
while abs(accuracy - previous_accuracy) >= 1 and epoch <= 20:
    print(f'------------- Epoch {epoch} -------------')

    # record previous accuracy
    previous_accuracy = accuracy

    # Iterate through training data
    for index in range(0, train.shape[0]):
        # Select a single image and associated y vector
        X = train[index:index+1,:].T
        y = Y[index:index+1].T

        # 1. Forward pass: compute Output/Prediction (o)
        h1 = calculate_layer_output(w1, X, b1, activation_type="relu")
        h2 = calculate_layer_output(w2, h1, b2, activation_type="relu")
        o = calculate_layer_output(w3, h2, b3, activation_type="softmax")

        # 2. Compute Loss Vector
        L = np.square(o - y)

        # 3. Backpropagation
        # Compute Loss derivative w.r.t. Output/Prediction vector (o)
        dL_do = 2.0 * (o - y)

        # Compute Output Layer derivatives
        dL3_dw3, dL3_dh2, dL3_db3 = layer_backprop(dL_do, o, h2, w3, "softmax")
        # Compute Hidden Layer 2 derivatives
        dL2_dw2, dL2_dh2, dL2_db2 = layer_backprop(dL3_dh2, h2, h1, w2, "relu")
        # Compute Hidden Layer 1 derivatives
        dL1_dw1, _, dL1_db1 = layer_backprop(dL2_dh2, h1, X, w1, "relu")

        # 4. Update weights & biases
        w1, b1 = gradient_descent(w1, b1, dL1_dw1, dL1_db1, learning_rate)
        w2, b2 = gradient_descent(w2, b2, dL2_dw2, dL2_db2, learning_rate)
        w3, b3 = gradient_descent(w3, b3, dL3_dw3, dL3_db3, learning_rate)


    # Compute & print Accuracy (%)
    accuracy = compute_accuracy(train, label, w1, b1, w2, b2, w3, b3)
    print(f'Accuracy: {accuracy:.2f} %')

    # Increment epoch
    epoch+=1

------------- Epoch 0 -------------
Accuracy: 35.46 %
------------- Epoch 1 -------------
Accuracy: 50.10 %
------------- Epoch 2 -------------
Accuracy: 56.46 %
------------- Epoch 3 -------------
Accuracy: 60.20 %
------------- Epoch 4 -------------
Accuracy: 63.30 %
------------- Epoch 5 -------------
Accuracy: 67.20 %
------------- Epoch 6 -------------
Accuracy: 70.09 %
------------- Epoch 7 -------------
Accuracy: 72.77 %
------------- Epoch 8 -------------
Accuracy: 75.17 %
------------- Epoch 9 -------------
Accuracy: 77.04 %
------------- Epoch 10 -------------
Accuracy: 69.76 %
------------- Epoch 11 -------------
Accuracy: 77.68 %
------------- Epoch 12 -------------
Accuracy: 79.26 %
------------- Epoch 13 -------------
Accuracy: 79.25 %


In [294]:
# Create empty list for predictions
test_predictions = []
# Iterate through test data
for image in test:
    # Forward pass: compute Output/Prediction (o)
    h1 = calculate_layer_output(w1, image.reshape(image.shape[0], 1)
                                                , b1, activation_type="relu")
    h2 = calculate_layer_output(w2, h1, b2, activation_type="relu")
    o = calculate_layer_output(w3, h2, b3, activation_type="softmax")

    # Add prediction to list
    test_predictions += [get_prediction(o)]

# Write test predictions to submission.csv file
submission = pd.DataFrame(test_predictions).reset_index().rename(
                                        columns={"index":"ImageId",0:"Label"})
submission['ImageId'] = submission['ImageId'] + 1
submission.to_csv('submission.csv', index=False)

In [330]:
# helper function to show image
from matplotlib import pyplot as plt
def plot_image(x):
    image = x.reshape(28, 28) * 255
    plt.gray()
    plt.imshow(image, interpolation='nearest')
    plt.show()
plot_image(train[8])